In [ ]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
splitfolders.ratio('/content/train/data', output="/content/dataset", seed = 432, ratio=(.6,0.25,0.15))

Copying files: 6252 files [00:00, 6571.08 files/s]


In [ ]:
import tensorflow as tf
from keras.models import Model
from tensorflow.keras import mixed_precision, layers
from keras.layers import Dense,GlobalAveragePooling2D,BatchNormalization,Conv2D,Conv2DTranspose,MaxPool2D,Input,LeakyReLU,Flatten,Reshape,GlobalMaxPooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import zipfile
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

In [ ]:
train_d = ImageDataGenerator(
        rescale=1./255)
SIZE=256
train_gen = train_d.flow_from_directory(
        '/content/dataset/train',
        target_size=(SIZE,SIZE),
        batch_size=32,
        class_mode="sparse",

)
test_gen = train_d.flow_from_directory(
        '/content/dataset/test',
        target_size=(SIZE,SIZE),
        batch_size=32,
        class_mode="sparse",
        shuffle=False
)

Found 3750 images belonging to 5 classes.
Found 940 images belonging to 5 classes.


In [ ]:
valid_gen = train_d.flow_from_directory(
        '/content/dataset/val',
        target_size=(SIZE,SIZE),
        batch_size=32,
        class_mode="sparse",
)

Found 1562 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
inception = InceptionV3(input_shape=(256,256,3), weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(inception.output)
x = Dense(2048,activation = 'relu')(x)
x=layers.Dropout(0.2)(x)
prediction = Dense(5, activation='softmax')(x)
model_inception = Model(inputs=inception.input, outputs=prediction)
for layer in model_inception.layers:
    layer.trainable = False
model_inception.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  optimizer= tf.keras.optimizers.Adam(
    learning_rate=0.0001,
),
  metrics=['accuracy']
)

r1 = model_inception.fit(
  train_gen,
  validation_data=valid_gen,
  epochs=15,
)

for layer in model_inception.layers[:160]:
   layer.trainable = False
for layer in model_inception.layers[160:]:
   layer.trainable = True
model_inception.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  optimizer= tf.keras.optimizers.Adam(
    learning_rate=0.0001,
),

  metrics=['accuracy']
)

r1 = model_inception.fit(
  train_gen,
  validation_data=valid_gen,
  epochs=5,)


Epoch 1/15
118/118 [==============================] - 21s 144ms/step - loss: 1.6818 - accuracy: 0.2000 - val_loss: 1.6533 - val_accuracy: 0.2061
Epoch 2/15
118/118 [==============================] - 16s 134ms/step - loss: 1.6780 - accuracy: 0.2000 - val_loss: 1.6533 - val_accuracy: 0.2061
Epoch 3/15
118/118 [==============================] - 16s 132ms/step - loss: 1.6845 - accuracy: 0.2032 - val_loss: 1.6533 - val_accuracy: 0.2061
Epoch 4/15
118/118 [==============================] - 16s 133ms/step - loss: 1.6769 - accuracy: 0.1984 - val_loss: 1.6533 - val_accuracy: 0.2061
Epoch 5/15
118/118 [==============================] - 15s 131ms/step - loss: 1.6867 - accuracy: 0.2061 - val_loss: 1.6533 - val_accuracy: 0.2061
Epoch 6/15
118/118 [==============================] - 15s 131ms/step - loss: 1.6827 - accuracy: 0.2083 - val_loss: 1.6533 - val_accuracy: 0.2061
Epoch 7/15
118/118 [==============================] - 16s 131ms/step - loss: 1.6800 - accuracy: 0.2075 - val_loss: 1.6533 - val_ac

In [ ]:
for layer in model_inception.layers[:172]:
   layer.trainable = False
for layer in model_inception.layers[172:]:
   layer.trainable = True
model_inception.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  optimizer= tf.keras.optimizers.Adam(
    learning_rate=0.0001,
),

  metrics=['accuracy']
)

r1 = model_inception.fit(
  train_gen,
  validation_data=valid_gen,
  epochs=10,)


Epoch 1/10
118/118 [==============================] - 31s 213ms/step - loss: 0.0321 - accuracy: 0.9912 - val_loss: 0.4711 - val_accuracy: 0.9251
Epoch 2/10
118/118 [==============================] - 24s 206ms/step - loss: 0.0260 - accuracy: 0.9917 - val_loss: 0.4697 - val_accuracy: 0.9136
Epoch 3/10
118/118 [==============================] - 24s 201ms/step - loss: 0.0441 - accuracy: 0.9877 - val_loss: 0.4288 - val_accuracy: 0.9155
Epoch 4/10
118/118 [==============================] - 24s 207ms/step - loss: 0.0441 - accuracy: 0.9893 - val_loss: 0.3349 - val_accuracy: 0.9347
Epoch 5/10
118/118 [==============================] - 24s 201ms/step - loss: 0.0134 - accuracy: 0.9968 - val_loss: 0.3127 - val_accuracy: 0.9341
Epoch 6/10
118/118 [==============================] - 24s 202ms/step - loss: 0.0209 - accuracy: 0.9939 - val_loss: 0.4761 - val_accuracy: 0.9149
Epoch 7/10
118/118 [==============================] - 24s 202ms/step - loss: 0.0181 - accuracy: 0.9944 - val_loss: 0.3262 - val_ac

In [ ]:
model_inception.evaluate(test_gen)

30/30 [==============================] - 3s 92ms/step - loss: 0.3895 - accuracy: 0.9319


[0.3894749879837036, 0.9319148659706116]

In [ ]:
y=model_inception.predict(test_gen)

30/30 [==============================] - 4s 89ms/step


In [ ]:
y=[np.argmax(x) for x in y]

In [ ]:
import math
number_of_examples = len(test_gen.filenames)
number_of_generator_calls = math.ceil(number_of_examples / (1.0 * 32))
# 1.0 above is to skip integer division
test_labels = []
for i in range(0,int(number_of_generator_calls)):
    test_labels.extend(np.array(test_gen[i][1]))

In [ ]:
acc=model_inception.evaluate(test_gen)

30/30 [==============================] - 3s 91ms/step - loss: 0.3895 - accuracy: 0.9319


In [ ]:
x=sklearn.metrics.cohen_kappa_score(y,test_labels)
print('kappa Score:',x)
print('Accuracy Score:',acc[1])

kappa Score: 0.9114388971080737
Accuracy Score: 0.9319148659706116


In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix
cf_cae=sklearn.metrics.confusion_matrix(test_labels, y)

In [ ]:
cf_cae

array([[301,   3,   1,   1,  12],
       [  3, 172,   0,   1,   0],
       [  6,   1, 131,   0,   0],
       [  8,   0,   0, 116,   1],
       [ 25,   2,   0,   0, 156]])

**PREDICTION**

In [ ]:
predict_gen = train_d.flow_from_directory(
        '/content/train/predict/pred',
        target_size=(SIZE,SIZE),
        batch_size=32,
        shuffle=False
)

Found 2680 images belonging to 1 classes.


In [ ]:
y_pred=model_inception.predict(predict_gen)

84/84 [==============================] - 9s 91ms/step


In [ ]:
y_pred =[np.argmax(x) for x in y_pred]

In [ ]:
len(y_pred)

2680

In [ ]:
pred_df=pd.read_csv('/content/sample_submission_ns2btKE.csv')

In [ ]:
pred_df['category']=y_pred

In [ ]:
pred_df.to_csv('/content/submission.csv')